In [21]:
import os
import google.generativeai as genai
from pinecone import Pinecone as pc
import dotenv

In [22]:
dotenv.load_dotenv()

#gemini api key
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY") 

#pinecone api key
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY") 

In [23]:
genai.configure(api_key=GEMINI_API_KEY)

In [4]:
EMBEDDING_MODEL = "models/gemini-embedding-001"  
PINECONE_INDEX_NAME = "medical-rag-index" 

In [5]:
print("Initializing Pinecone...")
pine_client= pc(
    api_key = os.getenv("PINECONE_API_KEY"), 
    )

Initializing Pinecone...


In [6]:
# Connect to your existing index
try:
    index = pine_client.Index(PINECONE_INDEX_NAME)
    print("\n--- Pinecone Index Stats ---")
    print(index.describe_index_stats())
    print("--------------------------\n")
except Exception as e:
    print(f"Error connecting to index '{PINECONE_INDEX_NAME}'. Please ensure it exists.")
    print(e)
    exit()


--- Pinecone Index Stats ---
{'dimension': 3072,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 9790}},
 'total_vector_count': 9790,
 'vector_type': 'dense'}
--------------------------



In [7]:
def retrieve_context(user_query, top_k=3):

    print(f"Received query: '{user_query}'")
    
    # 1. Embed the User's Query
  
    print("Embedding user query...")
    query_embedding = genai.embed_content(
        model=EMBEDDING_MODEL,
        content=user_query,
        task_type="RETRIEVAL_QUERY" 
    )['embedding']
    
    print(f"Query embedded successfully. Vector dimension: {len(query_embedding)}")

    # 2. Query Pinecone
    print(f"Querying Pinecone index for top {top_k} results...")
    query_results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True
    )
    
    # 3. Process and Return Results
    print("Processing results...")
    retrieved_context = []
    for match in query_results['matches']:
        context_text = match['metadata'].get('text', 'No text found in metadata.')
        retrieved_context.append(context_text)
        
     
        print(f"  - Match ID: {match['id']}, Score: {match['score']:.4f}")
      
        
    return retrieved_context

In [ ]:
if __name__ == "__main__":
    
    my_question = "on my face a lots of red pimple, tell me how danger is ?"
    
    # Retrieve the context
    context = retrieve_context(my_question, top_k=5)
   
    print("\n==================== RETRIEVED CONTEXT ====================\n")
    if context:
      
        print("\n---\n".join(context))
    else:
        print("No relevant context was found in Pinecone.")
    print("\n===========================================================")

Received query: 'on my face a lots of red pimple, tell me how danger is ?'
Embedding user query...
Query embedded successfully. Vector dimension: 3072
Querying Pinecone index for top 5 results...
Processing results...
  - Match ID: 5044, Score: 0.7193
  - Match ID: 5570, Score: 0.7138
  - Match ID: 4993, Score: 0.6845
  - Match ID: 5562, Score: 0.6839
  - Match ID: 5391, Score: 0.6688
['Question: What is (are) Rosacea ?\nAnswer: Rosacea is a long-term disease that affects your skin and sometimes your eyes. It causes redness and pimples. Rosacea is most common in women and people with fair skin. It most often affects middle-aged and older adults. In most cases, rosacea only affects the face. Symptoms can include - Frequent redness of the face, or flushing - Small, red lines under the skin - Acne - A swollen nose - Thick skin, usually on the forehead, chin, and cheeks - Red, dry, itchy eyes and sometimes vision problems No one knows what causes rosacea. You may be more likely to have it 